#### Notes for later

In [11]:
import polars as pl
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import time

## Pull from file / make dates

In [12]:
last_month = time.localtime().tm_mon - 1

current_yr = time.localtime().tm_year

# handling if last month was January
if last_month == 0:
    last_month = 12
    current_yr = current_yr - 1


# current_month = 4
# current_yr = 2025

df = pl.read_csv(r"C:\Users\Denne\OneDrive\Documents\GitHub\FinancialStreamline2\BankStreamline\Data\CreditHistory.csv", 
                 truncate_ragged_lines=True,)


# date work ========================================== DELETE IF THE CODE BELOW WORKS ==================================================================
# df = df.with_columns(pl.col("Post Date")
#                         .str.split("/")
#                         .alias("split_str")
#                         .list.to_struct()) \
#                         .unnest('split_str') \
#                         .rename({'field_0':'month', 'field_1':'day', 'field_2':'year'}) \
#                         .cast({'day':pl.Int32, 'month':pl.Int32, 'year':pl.Int32}) \
#         .filter((pl.col('month') == last_month) & 
#                 (pl.col('year') == current_yr)) \
#         .with_columns(date = pl.concat_str([pl.col('year').cast(pl.Utf8), pl.lit('-'), pl.col('month').cast(pl.Utf8), pl.lit('-'), pl.col('day').cast(pl.Utf8)], separator="")) \
#         .with_columns(monthName = pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.strftime("%b")) \
#         .with_columns(weekDay = pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.strftime("%a")) \
#         .with_columns(quarter = pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.quarter())

df = df.with_columns(
                pl.col("Post Date").str.strptime(pl.Date, "%m/%d/%Y").alias("date")
                ).with_columns([
                pl.col("date").dt.month().alias("month"),
                pl.col("date").dt.day().alias("day"),
                pl.col("date").dt.year().alias("year")
                ]) \
        .filter((pl.col('month') == last_month) & 
                (pl.col('year') == current_yr)) \
        .with_columns(date = pl.concat_str([pl.col('year').cast(pl.Utf8), pl.lit('-'), pl.col('month').cast(pl.Utf8), pl.lit('-'), pl.col('day').cast(pl.Utf8)], separator="")) \
        .with_columns(monthName = pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.strftime("%b")) \
        .with_columns(weekDay = pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.strftime("%a")) \
        .with_columns(quarter = pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.quarter())


        
# The gate/clean up        
df = df.filter(pl.col('Amount') < 0) \
        .drop('Category', 'Reference Number') \
        .rename({'Amount':'cost', 'Description': 'description'}) \
        .with_columns(cardType = pl.lit('Credit')) \
        .with_columns(cost = abs(pl.col('cost')))


df.limit(10)

Post Date,description,cost,date,month,day,year,monthName,weekDay,quarter,cardType
str,str,f64,str,i8,i8,i32,str,str,i8,str
"""10/31/2025""","""SONIC DRIVE IN #1948 870-8…",2.18,"""2025-10-31""",10,31,2025,"""Oct""","""Fri""",4,"""Credit"""
"""10/31/2025""","""WALMART.COM 800-9…",76.15,"""2025-10-31""",10,31,2025,"""Oct""","""Fri""",4,"""Credit"""
"""10/30/2025""","""USPS PO 0428080730 EL DO…",37.75,"""2025-10-30""",10,30,2025,"""Oct""","""Thu""",4,"""Credit"""
"""10/30/2025""","""MURPHY 1111 EL DO…",1.75,"""2025-10-30""",10,30,2025,"""Oct""","""Thu""",4,"""Credit"""
"""10/29/2025""","""MURPHY 1111 EL DO…",3.0,"""2025-10-29""",10,29,2025,"""Oct""","""Wed""",4,"""Credit"""
"""10/28/2025""","""NETFLIX.COM NETFL…",8.77,"""2025-10-28""",10,28,2025,"""Oct""","""Tue""",4,"""Credit"""
"""10/28/2025""","""WAL-MART #0530 EL DO…",7.11,"""2025-10-28""",10,28,2025,"""Oct""","""Tue""",4,"""Credit"""
"""10/28/2025""","""WALMART.COM 8009256278 BENTO…",1.0,"""2025-10-28""",10,28,2025,"""Oct""","""Tue""",4,"""Credit"""
"""10/28/2025""","""MURPHY 1111 EL DO…",3.29,"""2025-10-28""",10,28,2025,"""Oct""","""Tue""",4,"""Credit"""


# Categorize

In [13]:
df = df.with_columns(category = 
                  # ======================= Food =======================
    pl.when(pl.col('description').str.contains('McDon|CORPORATE|CHICK-FIL-A|SLIM CHICKENS')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('MADDIES PLACE|RAISING CANES|Subway|HICKORY')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('JAMBA|PIZZA|GOODCENTS|SONIC|TACO BELL|BUFFET')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('Waffle House|DAIRY QUEEN|COLDSTONE|MCGRAWS')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('LITTLE CAESARS|MCDON|WENDY|APPLEBEES|PERCY')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('EL SUR|BAREFOOT BISTRO|YAMATO|SMOOTHIE|CREAMERY')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('DOLLAR GENERAL|DOLLAR TREE|FLYING BURGER|WWW.HOMECHEF.IL')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('DOMINO.S|POPEYES|COCA COLA|LA VILLA MEXICAN|PJ\'S COFFEE')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('CRCKR BRRL|OFF THE RAIL CAFE|HOUSE-WYLIE|SWOLE FOOD')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('DC FUDDRUCKERS|THE BLACK CAT CAFE|RHEA LANA|BYUI FOOD')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('WHATABURGER|CUPBOP|TASTY DONUTS|HELLA FRESH|JOHNNY B\'S GRILL')).then(pl.lit('food'))
    .when(pl.col('description').str.contains('WILDCAT SNACK|ARBYS|SNACKS ABUELITA')).then(pl.lit('food'))
                  # ======================= subscriptions =======================
    .when(pl.col('description').str.contains('Adobe|Spotify|Phtoshp Lightrm|Peacock')).then(pl.lit('subscriptions'))
    .when(pl.col('description').str.contains('WMT PLUS|HEALTHWORKS|APPLE.COM/BILL')).then(pl.lit('subscriptions'))
    .when(pl.col('description').str.contains('NETFLIX.COM|NETFLIX|ADOBE|ADT SECURITY')).then(pl.lit('subscriptions'))
                  # ======================= wmt =======================
    .when(pl.col('description').str.contains('WM SUPER|Wal-Mart|WAL-MART|BROOKSHIRES|BROULIM|ALBERTSONS')).then(pl.lit('wmt'))
    .when(pl.col('description').str.contains('WALGR|SMITHS')).then(pl.lit('wmt'))
    .when(pl.col('description').str.contains('Walmart|WALMART') &
          ~(pl.col('description').str.contains('MURPHY'))).then(pl.lit('wmt'))
                  # ======================= gas =======================
    .when(pl.col('description').str.contains('MURPHY') & 
          ~pl.col('description').str.contains('DEPOSIT')).then(pl.lit('gas'))
    .when(pl.col('description').str.contains('SHELL|CHEVRON|CIRCLE K|LOVE\'S|EXXON EXPRESSWAY|EXXON MISSLE')).then(pl.lit('gas'))
    .when(pl.col('description').str.contains('MISSLE MART')).then(pl.lit('gas'))
                  # ======================= internet =======================
    .when(pl.col('description').str.contains('VIASAT')).then(pl.lit('internet'))
    .when(pl.col('description').str.contains('OPTIMUM')).then(pl.lit('internet'))
                  # ======================= shopping =======================
    .when(pl.col('description').str.contains('T J MAXX|OLD NAVY|SHEIN|REAL DEALS|SALLY BEAUTY')).then(pl.lit('shopping'))
    .when(pl.col('description').str.contains('LDS DIST ONLINE STORE|DC ULTA|DC H&amp|SEPHORA.COM')).then(pl.lit('shopping'))
    .when(pl.col('description').str.contains('American Eagle|SPORTSMANS WAREHOUSE|ZOE FRYE HAIR|OLDNAVY')).then(pl.lit('shopping'))
                  # ======================= amazon =======================
    .when(pl.col('description').str.contains('AMZN|AMAZON|Amazon.com|temu.com|eBay')).then(pl.lit('amazon'))
                  # ======================= fun =======================
    .when(pl.col('description').str.contains('Amazon Prime|YouTube|ZOO|MUSEUM|AQUARIUM|LION.S CLUB GOLF')).then(pl.lit('fun'))
    .when(pl.col('description').str.contains('COMFORT INN|CINEMA|HOLIDAY INN|El Dorado Golf|PlayStation')).then(pl.lit('fun'))
    .when(pl.col('description').str.contains('EXCALIBUR FAMILY FUN|AIRBNB|EXPEDIA|EL DORADO GOLF')).then(pl.lit('fun'))
                  # ======================= power =======================
    .when(pl.col('description').str.contains('ENTERGY')).then(pl.lit('power'))
                  # ======================= car =======================
    .when(pl.col('description').str.contains('O.REILLY|MUFFLEX MUFFLER|AUTOZONE|DC TAKE 5|KARL MALONE FORD')).then(pl.lit('car'))
    .when(pl.col('description').str.contains('VAN HOOK TIRE|IRONHEART AUTOMOTIVE')).then(pl.lit('car'))
                  # ======================= progressive =======================
    .when(pl.col('description').str.contains('PROG DIRECT|STATE FARM')).then(pl.lit('progressive'))
                  # ======================= water =======================
    .when(pl.col('description').str.contains('SHARE CHECK')).then(pl.lit('water'))
    .when(pl.col('description').str.contains('EL DORADO WATER UTI|EL DORADO WATER')).then(pl.lit('water'))
                  # ======================= tithing =======================
    .when(pl.col('description').str.contains('Ch JesusChrist  DONATION|Ch JesusChrist DONATION')).then(pl.lit('tithing'))
                  # ======================= Natural Gas =======================
    .when(pl.col('description').str.contains('SUMMIT')).then(pl.lit('natural gas'))
                  # ======================= Home improvement =======================
    .when(pl.col('description').str.contains('SHERWIN-WILLIAMS|THE HOME DEPOT|MAIN STREET ANTIQUES')).then(pl.lit('home improvement'))
    .when(pl.col('description').str.contains('HOBBYLOBBY|EVERYBODYS ANTIQUE|HOBBY-LOBBY')).then(pl.lit('home improvement'))
    .when(pl.col('description').str.contains('TIMMINS|Sherwin-Williams')).then(pl.lit('home improvement'))
                  # ======================= rent =======================
    .when((pl.col('description').str.contains('PENNYMAC'))).then(pl.lit('rent'))
    .otherwise(pl.lit('misc'))
)


controllables = ['food', 'wmt', 'shopping', 'subscriptions', 'fun', 'misc', 'amazon', 'food']
uncontrollables = ['tithing', 'progressive', 'power', 'gas', 'water', 'internet', 'rent', 'car']

df = df.with_columns(controllable = pl.when(pl.col('category').is_in(controllables)).then(pl.lit(1))
                                    .when(pl.col('category').is_in(uncontrollables)).then(pl.lit(0))
                                    .otherwise(pl.lit('error')))
                                    

df = df.select(['date', 'category', 'description', 'cost', 'controllable', 'monthName', 'month', 'day', 'year', 'weekDay', 'cardType', 'quarter'])

# df.limit(10)

# list(df.filter(pl.col('Category') == 'wmt')['description'].unique())
# list(df.filter(pl.col('Category') == 'gas')['description'].unique())
# list(df.filter(pl.col('Category') == 'unknown')['description'].unique())
df.sort(pl.col('cost'), descending=True).limit(20)

date,category,description,cost,controllable,monthName,month,day,year,weekDay,cardType,quarter
str,str,str,f64,str,str,i8,i8,i32,str,str,i8
"""2025-10-23""","""home improvement""","""The Sherwin-Williams Comp800-4…",101.39,"""error""","""Oct""",10,23,2025,"""Thu""","""Credit""",4
"""2025-10-31""","""wmt""","""WALMART.COM 800-9…",76.15,"""1""","""Oct""",10,31,2025,"""Fri""","""Credit""",4
"""2025-10-24""","""wmt""","""WALMART.COM 8009256278 800-9…",74.62,"""1""","""Oct""",10,24,2025,"""Fri""","""Credit""",4
"""2025-10-3""","""food""","""THE BLACK CAT CAFE EL DO…",58.0,"""1""","""Oct""",10,3,2025,"""Fri""","""Credit""",4
"""2025-10-5""","""wmt""","""WM SUPERCENTER #530 EL DO…",51.29,"""1""","""Oct""",10,5,2025,"""Sun""","""Credit""",4
…,…,…,…,…,…,…,…,…,…,…,…
"""2025-10-23""","""misc""","""MADISON PAYMENTS REXBU…",20.0,"""1""","""Oct""",10,23,2025,"""Thu""","""Credit""",4
"""2025-10-6""","""gas""","""MURPHY7520ATWALMART EL DO…",16.55,"""0""","""Oct""",10,6,2025,"""Mon""","""Credit""",4
"""2025-10-3""","""food""","""SLIM CHICKENS 12408 EL DO…",14.81,"""1""","""Oct""",10,3,2025,"""Fri""","""Credit""",4


### Look at last month's misc

In [14]:
list(df.filter(pl.col('category') == 'misc')['description'].unique())

['MADISON PAYMENTS         REXBURG      ID',
 'USPS PO 0428080730       EL DORADO    AR']

#### Make sure correct month/year is being loaded

In [15]:
print(list(df['month'].unique()))
print(list(df['monthName'].unique()))
print(list(df['weekDay'].unique()))
print(list(df['year'].unique()))
print(df.select(pl.col("date").min()))
print(df.select(pl.col("date").max())) # broken, looks at str, not date

[10]
['Oct']
['Thu', 'Mon', 'Tue', 'Fri', 'Wed', 'Sun']
[2025]
shape: (1, 1)
┌───────────┐
│ date      │
│ ---       │
│ str       │
╞═══════════╡
│ 2025-10-1 │
└───────────┘
shape: (1, 1)
┌───────────┐
│ date      │
│ ---       │
│ str       │
╞═══════════╡
│ 2025-10-8 │
└───────────┘


### Check number of rows (30ish per month)
Apr: 32
<br>May: 21
<br>June: 20
<br>July: 23
<br>Aug: 28
<br>Sep: 35
<br>Oct: 44

In [17]:
len(df)

44

### Load the data (handle w/ caution)

In [ ]:
# path = r'C:\Users\Denne\OneDrive\Documents\GitHub\FinancialStreamline2\BankStreamline\Data\BankDataProd.csv'


# with open(path, mode="a", encoding="utf-8") as f:
#    df.write_csv(f, include_header=False)